# Chapter 9 - Deep Learning–Based Recommender Systems

Deep learning techniques utilize recent and rapidly growing network architectures and optimization algorithms to train on large amounts of data and build more expressive and better-performing models. Graphics Processing Units (GPUs) and deep learning have been driving advances in recommender systems for the past few years. Due to their massively parallel architecture, using GPUs for computation provides higher performance and cost savings. Let’s first explore the basics of deep learning and then look at the deep learning–based collaborative filtering method (neural collaborative filtering)..

<div style="text-align:left;">
    <img src='images/NN.png' width='600'>
</div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
print(np.__version__)

1.26.4


In [3]:
import sys
print(sys.version)

3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]


In [4]:
# Importing libraries

import pandas as pd
import numpy as np

import sys
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset

#from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split,python_stratified_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

import warnings
warnings.filterwarnings("ignore")

System version: 3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version: 2.2.2
Tensorflow version: 2.13.1


In [5]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
#MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 256

SEED = 42

## 1. Load Dataset

In [6]:
#read csv data

df = pd.read_excel("data/Rec_sys_data.xlsx")
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850


In [7]:
df= df[['CustomerID','StockCode','Quantity','DeliveryDate']]

df["StockCode"] = df["StockCode"].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

df = df.dropna()

print(df.shape)
df.head()

(246706, 4)


,CustomerID,StockCode,Quantity,DeliveryDate
1,17850,71053.0,6,2010-12-02 08:26:00
2,17850,21730.0,6,2010-12-03 08:26:00
4,17850,22752.0,2,2010-12-04 08:26:00
7,17850,22633.0,6,2010-12-04 08:28:00
8,17850,22632.0,6,2010-12-03 08:28:00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246706 entries, 1 to 272403
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   CustomerID    246706 non-null  int64         
 1   StockCode     246706 non-null  float64       
 2   Quantity      246706 non-null  int64         
 3   DeliveryDate  246706 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 9.4 MB


In [9]:
#header=["userID", "itemID", "rating", "timestamp"]

df = df.rename(columns={'CustomerID':"userID",
                        'StockCode':"itemID",
                        'Quantity':"rating",
                        'DeliveryDate':"timestamp"})

df["userID"] = df["userID"].astype(int)
df["itemID"] = df["itemID"].astype(int)

df.head()

,userID,itemID,rating,timestamp
1,17850,71053,6,2010-12-02 08:26:00
2,17850,21730,6,2010-12-03 08:26:00
4,17850,22752,2,2010-12-04 08:26:00
7,17850,22633,6,2010-12-04 08:28:00
8,17850,22632,6,2010-12-03 08:28:00


## 2. Split the Data

In [10]:
# Split the data using the Spark chronological splitter provided in utilities

train, test = python_chrono_split(df, 0.75)

train_file = "data/train.csv"  # save train.csv in data file
test_file = "data/test.csv"    # save test.csv in data file

train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185041 entries, 37126 to 137323
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userID     185041 non-null  int32         
 1   itemID     185041 non-null  int32         
 2   rating     185041 non-null  int64         
 3   timestamp  185041 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(2), int64(1)
memory usage: 5.6 MB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61665 entries, 153935 to 137339
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   userID     61665 non-null  int32         
 1   itemID     61665 non-null  int32         
 2   rating     61665 non-null  int64         
 3   timestamp  61665 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(2), int64(1)
memory usage: 1.9 MB


## 3. Train the NCF Model

<div style="text-align:left;">
    <img src='images/nfc.png' width='800'>
</div>

`Collaborative filtering` methods have been the most popular for building recommendation systems in various domains. Popular techniques like `matrix
factorization` have been extensively used because they are easy to implement and provide accurate predictions. But in recent times, through new areas of research, deep learning– based models are being used increasingly in all domains, including collaborative filtering.

In [13]:
# Train the Neural Collaborative Filtering model on the training data, 
# and get the top-k recommendations for our testing data

data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

INFO:recommenders.models.ncf.dataset:Indexing data/train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing data/test.csv ...
INFO:recommenders.models.ncf.dataset:Indexing data/test_full.csv ...


In [14]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED)

In [15]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [8.27s]: train_loss = 0.310240 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [8.21s]: train_loss = 0.293256 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [8.21s]: train_loss = 0.286458 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [8.73s]: train_loss = 0.280940 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [8.15s]: train_loss = 0.277619 


Took 418.8465 seconds for training.


In [16]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 64.5342 seconds for prediction.


## 4. Evaluate How Well NCF Performs
The ranking metrics are used for evaluation.

In [17]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.035038
NDCG:	0.077844
Precision@K:	0.051487
Recall@K:	0.061386


In [18]:
all_predictions[all_predictions['userID']==12346].nlargest(5,'prediction')

,userID,itemID,timestamp,prediction
1995,12346,23245,NaT,0.996095
1918,12346,23167,NaT,0.983744
1494,12346,22720,NaT,0.977554
2042,12346,23307,NaT,0.976961
1212,12346,22423,NaT,0.976344


In [19]:
all_predictions = all_predictions.rename(columns={
    
    "userID":'CustomerID',
    "itemID":'StockCode',
    "rating":'Quantity',
    'prediction':'probability'
    
})

In [20]:
df_order = pd.read_excel('data/Rec_sys_data.xlsx', 'order')
df_customer = pd.read_excel('data/Rec_sys_data.xlsx', 'customer')
df_product = pd.read_excel('data/Rec_sys_data.xlsx', 'product')

## 5. Recommendation

In [21]:
def recommend_product(customer_id):

  print(" \n---------- Top 5 Bought StockCodes -----------\n")
  
  print(df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity'))

  top_5_bought = df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity')

 
    
  print('\n-------Product Name of bought StockCodes ------\n')

  print(df_product[df_product.StockCode.isin(top_5_bought.StockCode)]['Product Name'])


    
  print("\n --------- Top 5 Recommendations ------------ \n")

  print(all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability'))

  recommend = all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability')

    

  print('\n-------Product Name of Recommendations ------\n')

  print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])

In [22]:
recommend_product(13137)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12

-------Product Name of bought StockCodes ------

70      MightySkins Skin Decal Wrap Compatible with Li...
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
1353163       13137      20723       NaT     0.954715
1353164       13137      20724       NaT     0.948119
1353910       13137      21932       NaT     0.938771
1355093       13137      23204       NaT     0.931392
1354297       13137    

In [23]:
recommend_product(15127)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
272296       15127     23263        48
272287       15127     23354        24
272288       15127     22813        24
272289       15127     23096        24
272285       15127     21181        12

-------Product Name of bought StockCodes ------

13                billyboards Porcelain School Chalkboard
374     MightySkins Protective Vinyl Skin Decal for Po...
923     Zoan Synchrony Duo Sport Electric Snow Helmet ...
952     MightySkins Skin Decal Wrap Compatible with Sm...
1576    EMPIRE KLIX Klutch Designer Wallet Case for LG G2
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
4598711       15127      21791       NaT     0.910977
4598710       15127      21790       NaT     0.894973
4599482       15127      22713       NaT     0.872023
4599743       15127      22983       NaT     0.868161
4598854       15127    